# Dinámica de sistemas cuánticos

Consideremos un sistema compuesto de $N$ espines 1/2, descrito por el `Hamiltonian` ($\hbar = 1$):

\begin{equation}
\hat{H} = \sum_{i=1}^{N-1} J\hat{\sigma}^z_i \hat{\sigma}^z_{i+1} + \sum_{i=1}^N g\hat{\sigma}^x_i,
\end{equation}

donde el primer término corresponde a una interacción de vecinos cercanos en la dirección $z$, mientras que el segundo término genera dinámica coherente entre los espines que componen el sistema. El `Hamiltonian` anterior es el Hamiltoniano del modelo de Ising cuántico de ferromagnetismo en una dimensión.

Podemos construir el `Hamiltonian` denotado al inicio utilizando productos tensoriales de cada uno de los espacios de Hilbert que componen el sistema. Por ejemplo:
\begin{equation}
\
\hat{\sigma}^z_1 = \hat{\sigma}^z \otimes \mathbb{1} \otimes \mathbb{1} \otimes \cdots \otimes  \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1}, \\
\
\hat{\sigma}^z_2 = \mathbb{1} \otimes \hat{\sigma}^z \otimes \mathbb{1} \otimes \cdots \otimes  \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1}, \\
\
\hat{\sigma}^z_3 = \mathbb{1} \otimes \mathbb{1} \otimes \hat{\sigma}^z \otimes \cdots \otimes  \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1}, \\
\\
\vdots
\\
\hat{\sigma}^z_{N-2} = \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1} \otimes \cdots \otimes \hat{\sigma}^z \otimes \mathbb{1} \otimes \mathbb{1}, \\
\
\hat{\sigma}^z_{N-1} = \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1} \otimes \cdots \otimes  \mathbb{1} \otimes \hat{\sigma}^z \otimes \mathbb{1}, \\
\
\hat{\sigma}^z_{N} = \mathbb{1} \otimes \mathbb{1} \otimes \mathbb{1} \otimes \cdots \otimes  \mathbb{1} \otimes \mathbb{1} \otimes \hat{\sigma}^z \\
\
\end{equation}

Donde:

* $\otimes$ corresponde al producto tensorial.

* $\mathbb{1}$ a la matrix identidad $2x2$.

* $\hat{\sigma}^z$ es la matriz de Pauli en $z$ 

De esta forma se realizan $N-1$ productos tensoriales para el cálculo de cada $\hat{\sigma}^z_i$. Esto tambíen aplica para calcular los $\hat{\sigma}^x_i$

Con la librería `QuTip` obtener fácilmente las matrices de Pauli de $z$ y $x$, además nos permite realizar el producto tensorial entre los elmentos de un arreglo de forma ordenada.

In [30]:
import qutip as qt # Librería QuTip: La principal funcionalidad que ocupamos de esta librería es la 
                   # función "tensor", la cual realiza el producto tensorial entre los elementos de un arreglo
    
import numpy as np # Librería Numpy: Para hacer productos punto y sumatoria de matrices

Para que `QuTip` pueda realizar el producto tensorial, los objetos del arreglo deben ser de tipo `Qobj`. Por esto las matrices de Pauli y la matriz identidad debemos definirlas con funciones de `Qutip`:

In [31]:
sx = qt.sigmax() # Pauli X
sz = qt.sigmaz() # Pauli Z

iden = qt.qeye(2) # Identidad de tamaño 2

Y con éstas matrices podemos representar el `Hamiltoniano` utilizando el producto tensorial (`qt.tensor()`):

In [32]:
# Calcule el Hamiltoniano del model de Ising usando productos tensoriales de las matrices de Pauli

# Esta rutina debe devolver una matrix 2^Nx2^N que corresponde al modelo de Ising para N espines

def hamiltonian(J, g, N):
    
    sigma_z = [0] * N  # Vectores donde se van a guardar los sigma^z_i y los sigma^x_i
    sigma_x = [0] * N  # Se crean los arreglos de esta forma para evitar hacer "append"
    
    for i in range(N):
        
        sigma_zi = [0] * N  # Vectores donde se van a guardar las matrices 2x2 a las que se les va a
        sigma_xi = [0] * N  # aplicar el producto tensorial
        
        for j in range(i):
            
            sigma_zi[j] = (iden)  # Se guardan las matrices identidad que van antes de la i-ésima posición
            sigma_xi[j] = (iden)
        
        sigma_zi[i] = (sz)  # Se guarda la matriz de Pauli en la i-ésima posición
        sigma_xi[i] = (sx)
        
        for k in range(i+1,N):
                
            sigma_zi[k] = (iden) # Se guardan las matrices identidad que van después de la i-ésima posición
            sigma_xi[k] = (iden)
        
        sigma_z[i] = (qt.tensor(sigma_zi)) # Producto tensorial empezando en la primera posición hasta la
        sigma_x[i] = (qt.tensor(sigma_xi)) # última posición de los elementos del vector sigma_zi
        
    
    h_first = 0.0  # Primer término del hamiltoniano
    h_second = 0.0 # Segundo término del hamiltoniano
    
    for i in range(N-1):

        h_first += J*np.dot(sigma_z[i], sigma_z[i+1])

    for i in range(N):

        h_second += g*np.array(sigma_x[i])
    
    return np.real(h_first + h_second) # np.real es porque la parte imaginaria es 0 pero por alguna razón 
                                       # al pasar de un objeto "Qobj" a uno "nparray" añade los " + 0j "


La matriz es una $2^Nx 2^N$ que describe la dinámica interna del sistema:

In [33]:
N = 3

print(hamiltonian(2.0, 1.0, N))

[[ 4.  1.  1.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  1.  0.  0.]
 [ 1.  0. -4.  1.  0.  0.  1.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 0.  1.  0.  0.  1. -4.  0.  1.]
 [ 0.  0.  1.  0.  1.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  1.  1.  4.]]
